In [1]:
%matplotlib inline
import os, numpy as np, h5py, matplotlib.pyplot as plt
import math
import pandas as pd

import ase
import pyiron
from pyiron import Project, ase_to_pyiron
from ase.io import read,write
import spglib

from molmod.units import *
from molmod.constants import *

import tarfile
import pathlib
import sys

from ase import Atoms
from numpy import linalg as LA
from scipy.spatial.transform import Rotation as R

In [2]:
class FAmolecule():
    
    def __init__(self, at_c):
        self.atomlst = [at_c]
        self.at_c = at_c
        self.CoM = 0.0
    
    def add_atom(self,at):
        self.atomlst.append(at)
    
    def change_CoM(self):
        self.CoM = 0.0
        tot_mass = 0.0
        for at in self.atomlst:
            tot_mass +=at.mass 
            self.CoM += at.position*at.mass 
        self.CoM /= tot_mass 
        
    def random_rotation_FA(self):
        trans_vec = self.CoM
        rot_mat =  R.random().as_matrix()
        for at in self.atomlst:
            at.position -= trans_vec
            at.position = np.dot(rot_mat, at.position)
            at.position += trans_vec
    
def my_distance_func(at1, at2, cell):
    inv_cell = LA.inv(cell)
    dir_at1 = np.dot(at1.position,inv_cell)
    dir_at2 = np.dot(at2.position,inv_cell)
    dir_at2 -=np.around(dir_at2 - dir_at1)
    new_pos2 = np.dot(dir_at2,cell)
    return LA.norm(at1.position - new_pos2), new_pos2

def check_dist_FA(FAmol, atoms):
    for atFA in FAmol.atomlst:
        for at in atoms:
            dist = my_distance_func(atFA, at, atoms.cell)[0]
            if dist < 2.0:
                return False
    return True
    
def Get_FAmol_lst(atoms):
    FAmol_lst = []
    for at in atoms:
        if at.symbol == 'C':
            FAmol_lst.append(FAmolecule(at))
    for at in atoms:
        flag = False
        while at.symbol == 'N' and flag == False:
            for FAmol in FAmol_lst:
                dist, new_pos = my_distance_func(FAmol.at_c, at, atoms.cell)
                if dist < 2:
                    if dist >1.7:
                        print("long distance between C and N: " + str(dist))
                    at.position = new_pos
                    FAmol.add_atom(at)
                    flag = True
    for at in atoms:
        flag = False
        while at.symbol == 'H' and flag == False:
            for FAmol in FAmol_lst:
                for atFA in FAmol.atomlst:
                    if atFA.symbol != 'H':
                        dist, new_pos = my_distance_func(atFA, at, atoms.cell)
                        if dist < 1.56:
                            at.position = new_pos
                            FAmol.add_atom(at)
                            flag = True
    return FAmol_lst

def Get_new_FArot_atoms(atoms):
    
    atoms_c = atoms.copy()
    atoms_new = Atoms(cell=atoms_c.cell, pbc = True)
    for at in atoms_c:
        if at.symbol == 'Pb' or at.symbol == 'I':
            atoms_new.append(at)
    FAmol_lst = Get_FAmol_lst(atoms_c)
    
    for FAmol in FAmol_lst:
        assert len(FAmol.atomlst) == 8
        FAmol.change_CoM()
        FAmol.random_rotation_FA()
        attempt = 0
        threshold_attempts = 100
        while check_dist_FA(FAmol, atoms_new) == False and attempt < threshold_attempts:
            FAmol.random_rotation_FA() 
            attempt +=1
        if attempt > 0:
            print(attempt)
        if attempt == threshold_attempts:
            print("performed " + str(attempt) + " rotations for one molecule, restart again")
            return False
        for at in FAmol.atomlst:
            atoms_new.append(at)
            
    return atoms_new

In [3]:
"""
phase_dct = {"gamma":15500, "Csdelta":15550, "FAdelta":15600} #
for phase, vol_start in {"FAdelta":15600}.items(): #phase_dct.items():
    print("    "+phase)
    for vol in np.arange(0,3151,150):
        print("  " + str(int(vol+vol_start)))
        name_file = "vol_"+str(int(vol+vol_start))+".xyz"
        atoms = read(phase + "/"+ name_file)
        atoms_new = Get_new_FArot_atoms(atoms)
        while atoms_new == False:
            print("restarting again:")
            atoms_new = Get_new_FArot_atoms(atoms)

        write("RotatedMA/" + phase + "/" + name_file, atoms_new, format="extxyz")
"""        
phase_dct = {"gamma":15600, "Csdelta":15650, "FAdelta":15700} #
for phase, vol_start in {"FAdelta":15700}.items(): #phase_dct.items():
    print("    "+phase)
    for vol in np.arange(0,2401,600):
        print("  " + str(int(vol+vol_start)))
        name_file = "vol_"+str(int(vol+vol_start))+".xyz"
        atoms = read(phase + "/ValidationRuns/" + name_file)
        atoms_new = Get_new_FArot_atoms(atoms)
        while atoms_new == False:
            print("restarting again:")
            atoms_new = Get_new_FArot_atoms(atoms)

        write("RotatedMA/" + phase + "/ValidationRuns/" + name_file, atoms_new, format="extxyz")     


    FAdelta
  15700
long distance between C and N: 1.7620150760111397
long distance between: 1.8903946727289496
long distance between: 1.8080658931931068
long distance between: 1.8864772323242507
long distance between: 1.9440225934651822
long distance between: 1.9854824741588857
long distance between: 1.828884635590304
long distance between: 1.981842234627038
long distance between: 1.9246351262859227
long distance between: 1.92721584495793
long distance between: 1.8240329172125065
long distance between: 1.9029909150814572
long distance between: 1.9925316885377953
long distance between: 1.9650886091838378
long distance between: 1.9199006692086091
long distance between: 1.963893653944306
long distance between: 1.9010753090737116
long distance between: 1.8482472269042163
long distance between: 1.8780812351796792
long distance between: 1.981013769215545
long distance between: 1.7694674647743243
long distance between: 1.7823649586332053
long distance between: 1.9650319870756336
long distanc

In [4]:
phase = "Csdelta"
vol_start = 16200
vol = 150
name_file = "vol_"+str(int(vol+vol_start))+".xyz"
atoms = read(phase + "/" + name_file)
atoms_new = Get_new_FArot_atoms(atoms)
while atoms_new == False:
    print("restarting again:")
    atoms_new = Get_new_FArot_atoms(atoms)

27
1
4
5
1
1
10
2
5
3
1
1
2
96
1
1
1
5
1
1
1
3


In [23]:
atoms_tc = atoms

FAmol_lst = Get_FAmol_lst(atoms_tc)

for FAmol in FAmol_lst:
    FAmol_ind_lst = []
    for atFA in FAmol.atomlst:
        FAmol_ind_lst.append(atFA.index)
    for at in atoms_tc:
        if at.index not in FAmol_ind_lst:
            dist = my_distance_func(at, FAmol.at_c, atoms_tc.cell)[0]
            if dist < 5:
                for atFA in FAmol.atomlst:
                    dist = my_distance_func(at, atFA, atoms_tc.cell)[0]
                    if dist < 2.1:
                        print(atFA.index, at.index, at.symbol, dist)

#display(np.sort(atoms_new.get_distances(at.index,np.arange(768),mic=True))[:6])

76 636 I 2.028370897786893
80 148 H 1.8234402309466522
144 148 H 2.003770748628222
148 80 H 1.8234402309466533
148 144 H 2.0037707486282206
